In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
import re
warnings.filterwarnings('ignore')
#设置多行显示
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display

In [3]:
city_epidemic_data = pd.read_excel("../data/附件1.xlsx", sheet_name='城市疫情')
city_epidemic_data.head()

,日期,城市,新增确诊,新增治愈,新增死亡
0,2020-01-10,武汉,41,2,1
1,2020-01-11,武汉,0,4,0
2,2020-01-12,武汉,0,1,0
3,2020-01-15,武汉,0,5,1
4,2020-01-16,武汉,4,3,0


In [4]:
pivot_city_epidemic_data = city_epidemic_data.pivot_table(
    index=['城市', '日期'],
    values=['新增确诊', '新增治愈', '新增死亡'],
    aggfunc={'新增确诊': np.max, '新增治愈': np.max, '新增死亡': np.max}).fillna(0)
pivot_city_epidemic_data.head()

新增死亡  新增治愈  新增确诊
城市  日期                          
七台河 2020-01-26     0     0     1
    2020-01-27     0     0     1
    2020-01-30     0     0     1
    2020-01-31     0     0     2
    2020-02-01     0     0     1

In [5]:
# 查看有哪些城市
citys = []
for i in pivot_city_epidemic_data.index:
    citys.append(i[0])
citys = set(citys)
pivot_city_epidemic_data = pivot_city_epidemic_data.reset_index()
pivot_city_epidemic_data.head()
# pivot_city_epidemic_data.loc[pivot_city_epidemic_data['城市']=='武汉'].head()

,城市,日期,新增死亡,新增治愈,新增确诊
0,七台河,2020-01-26,0,0,1
1,七台河,2020-01-27,0,0,1
2,七台河,2020-01-30,0,0,1
3,七台河,2020-01-31,0,0,2
4,七台河,2020-02-01,0,0,1


In [7]:
# 处理单个城市：武汉
tabel_c = pivot_city_epidemic_data[pivot_city_epidemic_data['城市'] == '武汉']
print(tabel_c['日期'].iloc[0]) # 该城市疫情开始时间
start_day = tabel_c['日期'].iloc[0]
end_day = pd.to_datetime('2020/6/30') # 时间截至6月30日
tmp = pd.DataFrame()
while True:
    if end_day < start_day:
        break
    tmp_df = pd.DataFrame( 
        {"城市": ['武汉'], '日期': [start_day], '新增死亡': [0], '新增治愈': [0], '新增确诊': [0]}
    ) 
    tmp = tmp.append(tmp_df, ignore_index = True)
    start_day += pd.Timedelta(days=1)
tabel_c.head()

final_data = pd.DataFrame() # 任务1.1结果表
for i in range(len(tmp)):
    for j in range(len(tabel_c)):
        if tmp.loc[i, '日期'] == tabel_c['日期'].iloc[j]:
            tmp.loc[i, '新增死亡'] = tabel_c['新增死亡'].iloc[j]
            tmp.loc[i, '新增治愈'] = tabel_c['新增治愈'].iloc[j]
            tmp.loc[i, '新增确诊'] = tabel_c['新增确诊'].iloc[j]
pred_temp_list = [0, 0, 0]
post_temp_list = [0, 0, 0]
for i in range(len(tmp)):
    pred_temp_list[0] = tmp.iloc[i]['新增死亡']
    pred_temp_list[1] = tmp.iloc[i]['新增治愈']
    pred_temp_list[2] = tmp.iloc[i]['新增确诊']

    tmp['新增死亡'][i] += post_temp_list[0]
    tmp['新增治愈'][i] += post_temp_list[1]
    tmp['新增确诊'][i] += post_temp_list[2]

    post_temp_list[0] += pred_temp_list[0]
    post_temp_list[1] += pred_temp_list[1]
    post_temp_list[2] += pred_temp_list[2]
final_data = final_data.append(tmp, ignore_index = True)
final_data.columns = ['城市', '日期', '累积死亡', '累积治愈', '累积确诊']
final_data.head()

2020-01-10 00:00:00


,城市,日期,新增死亡,新增治愈,新增确诊
5387,武汉,2020-01-10,1,2,41
5388,武汉,2020-01-11,0,4,0
5389,武汉,2020-01-12,0,1,0
5390,武汉,2020-01-15,1,5,0
5391,武汉,2020-01-16,0,3,4


In [8]:
len(tmp)

173

In [ ]:
final_data = pd.DataFrame() # 任务1.1结果表
from tqdm import tqdm
for c in tqdm(citys): # 遍历城市
    tabel_c = pivot_city_epidemic_data[pivot_city_epidemic_data['城市'] == c]
#     print(tabel_c['日期'].iloc[0]) # 该城市疫情开始时间
    start_day = tabel_c['日期'].iloc[0]
    end_day = pd.to_datetime('2020/6/30') # 时间截至6月30日
    tmp = pd.DataFrame()
    while True:
        if end_day < start_day:
            break
        tmp_df = pd.DataFrame( 
            {"城市": [c], '日期': [start_day], '新增死亡': [0], '新增治愈': [0], '新增确诊': [0]}
        ) 
        tmp = tmp.append(tmp_df, ignore_index = True)
        start_day += pd.Timedelta(days=1)
    for i in range(len(tmp)):
        for j in range(len(tabel_c)):
            if tmp.loc[i, '日期'] == tabel_c['日期'].iloc[j]:
                tmp.loc[i, '新增死亡'] = tabel_c['新增死亡'].iloc[j]
                tmp.loc[i, '新增治愈'] = tabel_c['新增治愈'].iloc[j]
                tmp.loc[i, '新增确诊'] = tabel_c['新增确诊'].iloc[j]
    pred_temp_list = [0, 0, 0]
    post_temp_list = [0, 0, 0]
    for i in range(len(tmp)):
        pred_temp_list[0] = tmp.iloc[i]['新增死亡']
        pred_temp_list[1] = tmp.iloc[i]['新增治愈']
        pred_temp_list[2] = tmp.iloc[i]['新增确诊']

        tmp['新增死亡'][i] += post_temp_list[0]
        tmp['新增治愈'][i] += post_temp_list[1]
        tmp['新增确诊'][i] += post_temp_list[2]

        post_temp_list[0] += pred_temp_list[0]
        post_temp_list[1] += pred_temp_list[1]
        post_temp_list[2] += pred_temp_list[2]
    final_data = final_data.append(tmp, ignore_index = True)
final_data

  5%|███▉                                                                        | 23/444 [27:35:19<5:14:12, 44.78s/it]

In [10]:
final_data

,城市,日期,新增死亡,新增治愈,新增确诊
0,衢州,2020-01-22,0,0,1
1,衢州,2020-01-23,0,0,2
2,衢州,2020-01-24,0,0,2
3,衢州,2020-01-25,0,0,3
4,衢州,2020-01-26,0,0,5
...,...,...,...,...,...
68573,惠州,2020-06-26,0,62,62
68574,惠州,2020-06-27,0,62,62
68575,惠州,2020-06-28,0,62,62
68576,惠州,2020-06-29,0,62,62


In [11]:
final_data.columns = ['城市', '日期', '累积死亡', '累积治愈', '累积确诊']

In [12]:
final_data

,城市,日期,累积死亡,累积治愈,累积确诊
0,衢州,2020-01-22,0,0,1
1,衢州,2020-01-23,0,0,2
2,衢州,2020-01-24,0,0,2
3,衢州,2020-01-25,0,0,3
4,衢州,2020-01-26,0,0,5
...,...,...,...,...,...
68573,惠州,2020-06-26,0,62,62
68574,惠州,2020-06-27,0,62,62
68575,惠州,2020-06-28,0,62,62
68576,惠州,2020-06-29,0,62,62


In [18]:
final_data.to_csv("./task1.1.csv", encoding='gbk', index=False)

In [58]:
wuhan_data = final_data[final_data['城市']=='武汉']
dates = []
for i in range(1, 7):
    dates.append(pd.to_datetime('2020/'+ str(i) +'/15'))
# print(dates)
last_wuhan_data = pd.DataFrame()
for d in dates:
    last_wuhan_data = last_wuhan_data.append(wuhan_data[wuhan_data['日期']==d], ignore_index = True)
last_wuhan_data

,城市,日期,累积死亡,累积治愈,累积确诊
0,武汉,2020-01-15,2,12,41
1,武汉,2020-02-15,1233,3151,39462
2,武汉,2020-03-15,2469,38537,50003
3,武汉,2020-04-15,2579,46325,50008
4,武汉,2020-05-15,3869,46464,50339
5,武汉,2020-06-15,3869,46471,50340


In [64]:
shenzhen_data = final_data[final_data['城市']=='深圳']
dates = []
for i in range(1, 7):
    dates.append(pd.to_datetime('2020/'+ str(i) +'/15'))
# print(dates)
last_shenzhen_data = pd.DataFrame()
for d in dates:
    last_shenzhen_data = last_shenzhen_data.append(shenzhen_data[shenzhen_data['日期']==d], ignore_index = True)
last_shenzhen_data

,城市,日期,累积死亡,累积治愈,累积确诊
0,深圳,2020-02-15,0,115,414
1,深圳,2020-03-15,3,401,417
2,深圳,2020-04-15,3,414,419
3,深圳,2020-05-15,3,414,423
4,深圳,2020-06-15,3,414,423


In [69]:
baoding_data = final_data[final_data['城市']=='保定']
dates = []
for i in range(1, 7):
    dates.append(pd.to_datetime('2020/'+ str(i) +'/15'))
# print(dates)
last_baoding_data = pd.DataFrame()
for d in dates:
    last_baoding_data = last_baoding_data.append(baoding_data[baoding_data['日期']==d], ignore_index = True)
last_baoding_data

,城市,日期,累积死亡,累积治愈,累积确诊
0,保定,2020-02-15,0,19,32
1,保定,2020-03-15,0,32,32
2,保定,2020-04-15,0,32,32
3,保定,2020-05-15,0,32,32
4,保定,2020-06-15,0,32,36


In [73]:
last_wuhan_data

,城市,日期,累积死亡,累积治愈,累积确诊
0,武汉,2020-01-15,2,12,41
1,武汉,2020-02-15,1233,3151,39462
2,武汉,2020-03-15,2469,38537,50003
3,武汉,2020-04-15,2579,46325,50008
4,武汉,2020-05-15,3869,46464,50339
5,武汉,2020-06-15,3869,46471,50340


In [74]:
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ThemeType

list1 = [
    {"value": 41},
    {"value": 39462},
    {"value": 50003},
    {"value": 50008},
    {"value": 50340},
    {"value": 50340},
]

list2 = [
    {"value": 12},
    {"value": 3151},
    {"value": 38537},
    {"value": 46325},
    {"value": 46464},
    {"value": 46471},
]

list3 = [
    {"value": 2},
    {"value": 1233},
    {"value": 2469},
    {"value": 2579},
    {"value": 3869},
    {"value": 3869},
]


c = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_xaxis(['01-15', '02-15', '03-15', '04-15', '05-15', '06-15'])
    .add_yaxis("累积确诊人数", list1, category_gap="50%")
    .add_yaxis("累积治愈人数", list2, category_gap="50%")
    .add_yaxis("累积死亡人数", list3, category_gap="50%")
    .set_series_opts(show=False)
    .set_global_opts(title_opts=opts.TitleOpts(title="武汉每月15号统计结果"))
    .render("task1.1_bar-wuhan10.html")
)